In [2]:
try:
    import numpy as np
    import PIL
    from PIL import Image
    from numpy import asarray
    import time
    import copy
except:
    !pip install Pillow
    !pip install numpy
    import numpy as np
    import PIL
    from PIL import Image
    from numpy import asarray
    import time
    import copy

In [126]:


##time
time_cont = time.time()

##bat: 정사각형 한 칸의 픽셀길이
#m:세로
#n:가로
a=720
b=480
bat = 20
m = b // bat
n = a // bat
##다시 격자로 나눌 칸 정하기    
bbat = 6
p = m//bbat
q = n//bbat

##사진 가져오기
num = 'person'
#print(num,'.jpg')
image = Image.open('{}.jpg'.format(num))
image = image.resize((a, b))
#sharpened1 = image.filter(ImageFilter.SHARPEN);
#image = sharpened1.filter(ImageFilter.SHARPEN);
#print(image.format)
#print(image.size)
#print(image.mode)
#image.show()
##사진을 넘파이 형태로 변환
data = asarray(image)
#print(type(data))
#print(data.shape)

##사진을 n*m칸짜리 리스트로 변경
pool = []
for i in range(m):
    pool.append(list(range(n)))
for i in range(m):
    for j in range(n):
        pool[i][j] = ([round(np.mean(data[bat*i:bat*i+bat,bat*j:bat*j+bat,0])),round(np.mean(data[bat*i:bat*i+bat,bat*j:bat*j+bat,1])),round(np.mean(data[bat*i:bat*i+bat,bat*j:bat*j+bat,2]))])
        
##사전식 색구분 함수
devation = 46
def decide_color(dict,p,devat):
    
    #print(dict,p,devat)
    for i in dict:
        if abs(i[0]-p[0])<=devat and abs(i[1]-p[1])<=devat and abs(i[2]-p[2])<=devat:
            return dict, i[3]
    dict.append([p[0],p[1],p[2],len(dict)])
    #print(dict, alph[len(dict)])
    return dict, len(dict)-1
##pool의 각 칸에 색 부여
color_dictionary = []
cpool = []
for i in range(m):
    cpool.append(list(range(n)))
for i in range(m):
    for j in range(n):
        color_dictionary, cpool[i][j] = decide_color(color_dictionary,pool[i][j],devation)

##numpy로 변환
carray = np.asarray(cpool)        

##나눈 칸에서 가장 많은 값을 9로 변환
for i in range(p):
    for j in range(q):
        im = carray[i*bbat:i*bbat+bbat,j*bbat:j*bbat+bbat]
        im = np.reshape(im, (1, -1))
        #print(im[0])
        mmax = np.argmax(np.bincount(im[0]))
        #print(mmax)
        im = np.where(im == mmax, 9, im) #숫자 바꾸는거 있지 말기
        #print(im)
        carray[i*bbat:i*bbat+bbat,j*bbat:j*bbat+bbat] = np.reshape(im, (bbat, -1))
     
preg = []
same = False
R=-1
two_over = [False,-1]
thr_over = [False, -1]
for i in range(m):
    preg.append([])
    if list(carray[i]) != [9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9]:
        for j in range(n):
            if two_over[0] == True or thr_over[0] == True:
                if two_over[1] == j or thr_over[1] == j:
                    two_over[0] = False
                    thr_over[0] = False
                else:
                    continue
            if carray[i,j] != 9 and same == False:
                preg[i].append([j,-1])
                R+=1
                same = True
                if j == n-1:
                    preg[i][R] = [j,j]
                elif carray[i,j+1] == 9:
                    preg[i][R] = [j,j]
            elif carray[i,j]!=9:
                preg[i][R] = [preg[i][R][0],j]
            elif j < n-2 and carray[i,j+2] !=9:
                two_over = [True, j+2]
            #elif j < n-3 and carray[i,j+3] !=9:
            #    two_over = [True, j+3]
            else:
                same = False
            
            #print(i,j,preg[i])
        R = -1
        same = False
#오래걸림 => 차나리 조각을 정리하면서 갈까?
#조각 경로 찾기
sagment = []
asa = True
for i in range(m):
    if preg[i] != []:
        sag = copy.deepcopy(sagment)
        for pn in range(len(preg[i])):
            for sn in range(len(sag)):
                #print(pn,sn,i, preg[i], sag)
                #print(preg[i][pn],sag[sn][-1]) # set(preg[i][pn]) & set(sag[sn][-1]) != set():
                if sag[sn][-1] != [] and (sag[sn][-1][0] <= preg[i][pn][1] and sag[sn][-1][1] >= preg[i][pn][0]):
                    if len(sagment[sn]) == i:
                        sagment[sn].append(preg[i][pn]+[i])
                    else:
                        #print(preg[i][pn], type(preg[i][pn]))
                        sagment.append(sagment[sn][0:-1]+[preg[i][pn]+[i]])
                    asa = False
                elif pn != len(preg[i])-1 and sag[sn][-1] != [] and (sag[sn][-1][0] <= preg[i][pn+1][1] and sag[sn][-1][1] >= preg[i][pn+1][0]):
                    if len(sagment[sn]) == i:
                        sagment[sn].append(sag[sn][-1])
                    else:
                        print(preg[i][pn], type(preg[i][pn]))
                        sagment.append(sagment[sn][0:-1]+[sag[sn][-1]])
                #else:
                #    sag[sn].append([])
            if asa == True:
                sagment.append([])
                for asdf in range(i):
                    sagment[-1].append([])
                sagment[-1].append(preg[i][pn]+[i])
            asa = True
        for sn in range(len(sagment)):
            if len(sagment[sn]) != i+1:
                sagment[sn].append([])
            #print(len(sagment[sn]),sagment[sn], end = " ")
        #print("")
        #print("")
    #print(("working time = {}".format(time.time() - time_cont)))
        
for i in range(len(sagment)):
    sagment[i] = list(map(tuple, sagment[i]))
    
div = [set(sagment[0])]
del sagment[0]

#조각 모음집
#print(sagment)
while True:
    im = sagment[0]
    for i in range(len(div)):
        if sagment == []:
            break
        a = div[i] & set(sagment[0])
            
        if len(a) > 3:
            div[i] = div[i] | set(sagment[0])
            del sagment[0]
            #print("?")
        #print("!")
    if sagment != [] and im == sagment[0]:
        #print("!!")
        div.append(set(sagment[0]))
        del sagment[0]
                    
    if sagment == []:
        break

#조각 모은거 정리
past = len(div)
now = -1
#print(div)
while past != now:
    past = len(div)
    for asdf in range(1, len(div)):
        for i in range(1, len(div)):
            if i<len(div) and div[0]&div[i] != {()}:
                print("!")
                div[0] = div[0] | div[i]
                del div[i]
        if len(div) == 1:
            break
        div = div[1:] + [div[0]]
    now = len(div)
    
for i in range(len(div)):
    div[i] = list(map(list, div[i]))
    
res = []
for i in range(len(div)-1,-1,-1):
    if i <= len(div) and len(div[i]) > 4:
        try:
            div[i].remove([])
        except:
            pass
        im = np.asarray(div[i]).T
        #print(im)
        res.append([max(0,np.min(im[0])-1),min(n,np.max(im[1])+1), min(m,np.max(im[2])+1),max(0,np.min(im[2])-1)]) # 좌우하상
print(("working time = {}".format(time.time() - time_cont)))  





!
!
!
!
working time = 0.33510422706604004


In [127]:
cropped_img = []
for i in range(len(res)):
    area = (res[i][0]*bat, res[i][3]*bat, res[i][1]*bat, res[i][2]*bat)
    cropped_img.append(image.crop(area))

image.show()
for i in range(len(cropped_img)):
    cropped_img[i].show()

In [123]:
for i in range(len(carray)):
    print(carray[i])

[9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9]
[9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9]
[9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 1]
[9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 2 1 1 1 1 1 3]
[0 0 0 0 3 3 9 4 5 9 0 9 9 9 3 3 9 9 9 9 3 3 3 3]
[9 9 9 9 9 9 9 5 6 9 9 9 9 9 9 3 9 9 2 1 0 0 9 9]
[9 9 9 9 9 9 9 6 6 7 9 9 9 9 9 9 1 2 8 8 8 0 9 9]
[9 9 9 9 9 9 9 5 6 6 9 9 9 9 9 9 1 0 0 2 0 9 9 9]
[9 9 9 9 9 9 9 4 6 9 9 9 9 9 9 9 2 2 9 9 9 3 9 9]
[ 9  9  9  9  9  9  9  3 10 10  3  9  9  9  9  9  2  9  9  9  9  3  3  3]
[ 9  9  3 11 10 11  3  3 10 11  9  9  9  9  9  9  9  9  9  9  9  9  9  3]
[ 9  9  9 11 11 10 10 11 10 11  9  9  9  9  9  9  9  9  3  3  3  9  9  9]
[ 9  9  3  3  9  9  3  3 10  3  9  9  9  9  3  9  9  3  3  3  9  9  3  3]
[ 9  9  9  9  9  3  3  3 11  3  3  3  3  9  9  9  9  9  9  9  9  9  9  9]
[9 9 9 9 9 9 9 9 3 9 9 9 9 9 9 9 9 9 9 9 9 9 9 3]
[9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9]


In [124]:
cpool

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 1, 1, 1, 1, 1, 3],
 [0, 0, 0, 0, 3, 3, 1, 4, 5, 1, 0, 1, 1, 1, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3],
 [1, 1, 1, 1, 1, 1, 1, 5, 6, 1, 1, 1, 1, 1, 1, 3, 3, 3, 2, 1, 0, 0, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 6, 6, 7, 1, 1, 1, 1, 1, 1, 1, 2, 8, 8, 8, 0, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 5, 6, 6, 1, 1, 1, 1, 1, 1, 1, 0, 0, 2, 0, 1, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 4, 6, 9, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 3, 1, 1],
 [1, 1, 1, 1, 1, 1, 1, 3, 10, 10, 3, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 3, 3, 3],
 [1, 1, 3, 11, 10, 11, 3, 3, 10, 11, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 3],
 [1,
  1,
  1,
  11,
  11,
  10,
  10,
  11,
  10,
  11,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  3,
  3,
  3,
  1,
  1,
  1],
 [1, 1, 3, 3, 1, 1, 3, 3, 10, 3, 1, 1, 1

In [18]:
sag

[[[],
  [],
  [],
  [],
  [13, 14, 4],
  [12, 14, 5],
  [9, 17, 6],
  [9, 17, 6],
  [10, 13, 8],
  [10, 13, 8],
  [12, 12, 10],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  []],
 [[],
  [],
  [],
  [],
  [],
  [30, 35, 5],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  []],
 [[],
  [],
  [],
  [],
  [],
  [],
  [26, 29, 6],
  [26, 29, 6],
  [26, 29, 6],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  []],
 [[],
  [],
  [],
  [],
  [],
  [],
  [],
  [0, 2, 7],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  []],
 [[],
  [],
  [],
  [],
  [13, 14, 4],
  [12, 14, 5],
  [9, 17, 6],
  [10, 17, 7],
  [10, 13, 8],
  [10, 13, 8],
  [12, 12, 10],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  []],
 [[],
  [],
  [],
  [],
  [],
  [],
  [26, 29, 6],
  [28, 29, 7],
  [28, 29, 7],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  [],
  

In [5]:


##time
time_cont = time.time()

##bat: 정사각형 한 칸의 픽셀길이
#m:세로
#n:가로
a=720
b=480
bat = 30
m = b // bat
n = a // bat
##다시 격자로 나눌 칸 정하기    
bbat = 4
p = m//bbat
q = n//bbat

##사진 가져오기
num = 'person'
#print(num,'.jpg')
image = Image.open('{}.jpg'.format(num))
image = image.resize((a, b))
#sharpened1 = image.filter(ImageFilter.SHARPEN);
#image = sharpened1.filter(ImageFilter.SHARPEN);
#print(image.format)
#print(image.size)
#print(image.mode)
#image.show()
##사진을 넘파이 형태로 변환
data = asarray(image)
#print(type(data))
#print(data.shape)

##사진을 n*m칸짜리 리스트로 변경
pool = []
for i in range(m):
    pool.append(list(range(n)))
for i in range(m):
    for j in range(n):
        pool[i][j] = ([round(np.mean(data[bat*i:bat*i+bat,bat*j:bat*j+bat,0])),round(np.mean(data[bat*i:bat*i+bat,bat*j:bat*j+bat,1])),round(np.mean(data[bat*i:bat*i+bat,bat*j:bat*j+bat,2]))])
        
##사전식 색구분 함수
devation = 40
def decide_color(dict,p,devat):
    
    #print(dict,p,devat)
    for i in dict:
        if abs(i[0]-p[0])<=devat and abs(i[1]-p[1])<=devat and abs(i[2]-p[2])<=devat:
            return dict, i[3]
    dict.append([p[0],p[1],p[2],len(dict)])
    #print(dict, alph[len(dict)])
    return dict, len(dict)-1
##pool의 각 칸에 색 부여
color_dictionary = []
cpool = []
for i in range(m):
    cpool.append(list(range(n)))
for i in range(m):
    for j in range(n):
        color_dictionary, cpool[i][j] = decide_color(color_dictionary,pool[i][j],devation)

##numpy로 변환
carray = np.asarray(cpool)        

##나눈 칸에서 가장 많은 값을 9로 변환
for i in range(p):
    for j in range(q):
        im = carray[i*bbat:i*bbat+bbat,j*bbat:j*bbat+bbat]
        im = np.reshape(im, (1, -1))
        #print(im[0])
        mmax = np.argmax(np.bincount(im[0]))
        #print(mmax)
        im = np.where(im == mmax, 9, im)
        #print(im)
        carray[i*bbat:i*bbat+bbat,j*bbat:j*bbat+bbat] = np.reshape(im, (bbat, -1))
     
preg = []
same = False
R=-1
two_over = [False,-1]
thr_over = [False, -1]
for i in range(m):
    preg.append([])
    if list(carray[i]) != [9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9, 9]:
        for j in range(n):
            if two_over[0] == True or thr_over[0] == True:
                if two_over[1] == j or thr_over[1] == j:
                    two_over[0] = False
                    thr_over[0] = False
                else:
                    continue
            if carray[i,j] != 9 and same == False:
                preg[i].append([j,-1])
                R+=1
                same = True
                if j == n-1:
                    preg[i][R] = [j,j]
                elif carray[i,j+1] == 9:
                    preg[i][R] = [j,j]
            elif carray[i,j]!=9:
                preg[i][R] = [preg[i][R][0],j]
            elif j < n-2 and carray[i,j+2] !=9:
                two_over = [True, j+2]
            #elif j < n-3 and carray[i,j+3] !=9:
            #    two_over = [True, j+3]
            else:
                same = False
            
            #print(i,j,preg[i])
        R = -1
        same = False
        
#오래걸림 => 차나리 조각을 정리하면서 갈까?
#조각 경로 찾기 밑으로
restsag = []
sagment = []
asa = True
for i in range(m):
    if preg[i] != []:
        sag = copy.deepcopy(sagment)
        for pn in range(len(preg[i])):
            for sn in range(len(sag)):#믿으로 찾자 -> 있으면 -1값이 []가 아님
                if sag[sn][-1] != [] and (sag[sn][-1][0] <= preg[i][pn][1] and sag[sn][-1][1] >= preg[i][pn][0]):
                    if len(sagment[sn]) == i:
                        sagment[sn].append(preg[i][pn]+[i])
                    else:
                        sagment.append(sagment[sn][0:-1]+[preg[i][pn]+[i]])
                    asa = False
                elif pn != len(preg[i])-1 and sag[sn][-1] != [] and (sag[sn][-1][0] <= preg[i][pn+1][1] and sag[sn][-1][1] >= preg[i][pn+1][0]):
                    if len(sagment[sn]) == i:
                        sagment[sn].append(sag[sn][-1])
                    else:
                        #print(preg[i][pn], type(preg[i][pn]))
                        sagment.append(sagment[sn][0:-1]+[sag[sn][-1]])
            
            if asa == True:  #시작점
                sagment.append([])
                for asdf in range(i):
                    sagment[-1].append([])
                sagment[-1].append(preg[i][pn]+[i])
            asa = True
            
        for sn in range(len(sagment)-1,-1,-1): #끝난 값들은 반복되지 않도록
            if len(sagment[sn]) != i+1:
                restsag.append(sagment[sn])
                del sagment[sn]
                
    #print(("working time = {}".format(time.time() - time_cont)))
sagment = sagment + restsag
#print(sagment)
for i in range(len(sagment)):
    sagment[i] = list(map(tuple, sagment[i]))
#print(sagment)
div = [set(sagment[0])]
restsag = sagment[0]
del sagment[0]

#조각 모음집
#print(sagment)
im = restsag
while True:
    for i in range(len(div)):
        if sagment == []:
            break
        a = div[i] & set(sagment[0])
            
        if len(a) > 3:
            div[i] = div[i] | set(sagment[0])
            del sagment[0]
            #print("?")
        #print("!")
    if sagment != [] and im == sagment[0]:
        #print("!!")
        div.append(set(sagment[0]))
        del sagment[0]
                    
    if sagment == []:
        break
    im = sagment[0]

#조각 모은거 정리
past = len(div)
now = -1
#print(div)
while past != now:
    past = len(div)
    for asdf in range(1, len(div)):
        for i in range(1, len(div)):
            if i<len(div) and div[0]&div[i] != {()}:
                #print("!")
                div[0] = div[0] | div[i]
                del div[i]
        if len(div) == 1:
            break
        div = div[1:] + [div[0]]
    now = len(div)
    
for i in range(len(div)):
    div[i] = list(map(list, div[i]))
    
res = []
for i in range(len(div)-1,-1,-1):
    if (i <= len(div) and len(div[i])> 4):
        try:
            div[i].remove([])
        except:
            pass
        im = np.asarray(div[i]).T
        #print(im)
        res.append([max(0,np.min(im[0])-3),min(n,np.max(im[1])+3), min(m,np.max(im[2])+3),max(0,np.min(im[2])-3)]) # 좌우하상
        
         
if res == []:
    for i in range(len(div)-1,-1,-1):
        try:
            div[i].remove([])
        except:
            pass
        im = np.asarray(div[i]).T
        res.append([max(0,np.min(im[0])-3),min(n,np.max(im[1])+3), min(m,np.max(im[2])+3),max(0,np.min(im[2])-3)]) # 좌우하상
print(("working time = {}".format(time.time() - time_cont)))  





working time = 0.04491281509399414


In [6]:
cropped_img = []
for i in range(len(res)):
    area = (res[i][0]*bat, res[i][3]*bat, res[i][1]*bat, res[i][2]*bat)
    cropped_img.append(image.crop(area))

image.save("res\{}_orgin.jpg".format(num),'JPEG')
for i in range(len(cropped_img)):
    cropped_img[i].save("res\{}{}.jpg".format(num,i),'JPEG',encoding = "UTF-8" )

In [240]:
len(cropped_img)

3

In [216]:
for i in range(len(cpool)):
    print(carray[i])

[9 9 9 9 9 9 9 9 9 9 9 1 2 2 2 1 2 2 2 2 2 3 3 3]
[9 9 9 9 9 9 9 9 9 9 9 9 9 4 4 4 5 1 1 1 2 2 1 2]
[9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 4 9 9 9 9 9]
[9 9 9 9 4 1 9 9 9 9 9 9 9 9 9 9 9 4 1 9 9 9 9 4]
[9 9 9 9 9 2 9 9 1 9 9 9 9 9 9 9 9 1 9 9 1 1 9 9]
[9 9 9 9 9 5 9 9 1 9 9 9 9 9 9 9 9 1 9 9 9 4 9 9]
[9 9 9 9 9 9 9 9 1 9 9 9 9 9 9 9 1 1 4 9 9 9 9 9]
[9 9 9 9 0 9 5 9 1 9 9 9 9 9 9 4 2 2 1 9 9 9 9 9]
[9 9 9 9 0 5 5 1 1 9 9 9 9 9 9 9 1 2 4 9 9 9 9 9]
[9 9 9 9 9 9 5 9 4 9 9 9 9 9 9 9 1 1 5 9 9 9 9 9]
[9 9 9 9 9 9 0 9 9 9 9 9 9 9 9 9 5 1 5 9 9 9 9 9]
[9 9 9 9 9 0 0 9 9 9 9 9 9 9 9 9 4 4 4 9 9 9 9 9]
[9 9 9 9 4 9 9 9 9 9 9 9 9 9 9 9 9 4 9 9 9 9 9 9]
[9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9]
[9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9]
[9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9 9]


In [215]:
div

[[[4, 4, 12]],
 [[13, 23, 1], [11, 23, 0]],
 [[18, 18, 2]],
 [[23, 23, 3]],
 [[17, 18, 3]],
 [[4, 5, 3]],
 [[17, 21, 4]],
 [[5, 8, 4]],
 [[21, 21, 5]],
 [[17, 17, 5]],
 [[5, 8, 5]],
 [[16, 18, 6]],
 [[8, 8, 6]],
 [[15, 18, 7]],
 [[8, 8, 7]],
 [[6, 6, 7]],
 [[4, 4, 7]],
 [[16, 18, 8]],
 [[4, 8, 8]],
 [[16, 18, 9]],
 [[8, 8, 9]],
 [[6, 6, 9]],
 [[16, 18, 10]],
 [[6, 6, 10]],
 [[16, 18, 11]],
 [[5, 6, 11]],
 [[17, 17, 12]]]